In [15]:
#imports
import os
import requests
import pandas as pd
import numpy as np

In [16]:
#api key for guardian loaded from evniromental variable

#%env GUARDIAN_KEY
GUARDIAN_KEY = os.getenv('GUARDIAN_KEY')

#guardian news api endpoint sorted by newest
url = 'https://content.guardianapis.com/news?order-by=newest&api-key=test'

In [17]:
#send a reguest
r = requests.get(url)
r.status_code

200

In [18]:
#read json response to news variable, and then to dataframe
news = r.json()
news = pd.DataFrame(news['response']['results'])

In [19]:
#links for acticles are in webUrl column
url_list = news.webUrl

In [20]:
url_list[0]

'https://www.theguardian.com/news/2023/feb/26/corrections-and-clarifications'